# Evaluate Pretrained `super-image` Models

---

[Github](https://github.com/eugenesiow/super-image) | All Models @ [huggingface.co](https://huggingface.co/models?filter=super-image) | All Datasets @ [huggingface datasets](https://huggingface.co/datasets?filter=task_ids:other-other-image-super-resolution)

---

Notebook to evaluate pretrained `super-image` models with common image super resolution datasets.

The notebook is structured as follows:
* Setting up the Environment
* Loading the dataset
* Evaluating the Model (Running Inference)

## Setting up the Environment

#### Install the library

We will install the `super-image` and huggingface `datasets` library using `pip install`.

In [25]:
!pip install -qq datasets super-image

## Loading the dataset

We download the [`Set5`]() dataset using the huggingface `datasets` library. You can explore more super resolution datasets [here](https://huggingface.co/datasets?filter=task_ids:other-other-image-super-resolution).

In [26]:
from datasets import load_dataset

dataset = load_dataset('eugenesiow/Set5', 'bicubic_x2', split='validation')

RuntimeError: Dataset scripts are no longer supported, but found Set5.py

The following code will show the the first image (high resolution and the low resolution (half sized) images) from the dataset in the notebook.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

cv2_imshow(cv2.imread(dataset[0]["hr"]))
cv2_imshow(cv2.imread(dataset[0]["lr"]))

## Evaluating the Model (Running Inference)

To evaluate the a model for the [PSNR](https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio#Quality_estimation_with_PSNR) and [SSIM](https://en.wikipedia.org/wiki/Structural_similarity#Algorithm) metrics we run the following code:

* `EvalDataset(dataset)` converts the dataset to an evaluation dataset that can be fed in to a PyTorch dataloader.
* `EdsrModel.from_pretrained` - Download and load a small, pre-trained deep-learning model to the `model` variable. You can replace this with [other](https://huggingface.co/models?filter=super-image) pretrained models.
* `EvalMetrics().evaluate(model, eval_dataset)` - Run the evaluation on the `eval_dataset` using the `model`.

In [ ]:
from super_image import EdsrModel
from super_image.data import EvalDataset, EvalMetrics

eval_dataset = EvalDataset(dataset)
model = EdsrModel.from_pretrained('eugenesiow/edsr-base', scale=2)
EvalMetrics().evaluate(model, eval_dataset)

We can see from the output that the PSNR for this model on this dataset is `38.02` and the SSIM is `0.9607`.